In [ ]:
from initialise import *

with open('modelling_data.pickle', 'rb') as file:
    modelling_data = pickle.load(file)

In [ ]:
# Confusion Matrix

In [ ]:
# ROC-AUC

In [ ]:
# Feature Importance Plot

In [ ]:
# Shapley Values